In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [66]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

metadata = pd.read_csv('/content/drive/MyDrive/mamoriya/dataset_simulations/summary.csv')
metadata[['block_dimensions_a', 'block_dimensions_b', 'block_dimensions_c']] = metadata['block_dimensions'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))
metadata[['block_position_a', 'block_position_b', 'block_position_c']] = metadata['block_position'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))
metadata[['block_velocity_a', 'block_velocity_b', 'block_velocity_c']] = metadata['block_velocity'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))
metadata[['tank_dimensions_a', 'tank_dimensions_b', 'tank_dimensions_c']] = metadata['tank_dimensions'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

metadata['block_dimensions_a'] = pd.to_numeric(metadata['block_dimensions_a'], errors='coerce').astype(float)
metadata['block_dimensions_b'] = pd.to_numeric(metadata['block_dimensions_b'], errors='coerce').astype(float)
metadata['block_dimensions_c'] = pd.to_numeric(metadata['block_dimensions_c'], errors='coerce').astype(float)
metadata['block_position_a'] = pd.to_numeric(metadata['block_position_a'], errors='coerce').astype(float)
metadata['block_position_b'] = pd.to_numeric(metadata['block_position_b'], errors='coerce').astype(float)
metadata['block_position_c'] = pd.to_numeric(metadata['block_position_c'], errors='coerce').astype(float)
metadata['block_velocity_a'] = pd.to_numeric(metadata['block_velocity_a'], errors='coerce').astype(float)
metadata['block_velocity_b'] = pd.to_numeric(metadata['block_velocity_b'], errors='coerce').astype(float)
metadata['block_velocity_c'] = pd.to_numeric(metadata['block_velocity_c'], errors='coerce').astype(float)
metadata['tank_dimensions_a'] = pd.to_numeric(metadata['tank_dimensions_a'], errors='coerce').astype(float)
metadata['tank_dimensions_b'] = pd.to_numeric(metadata['tank_dimensions_b'], errors='coerce').astype(float)
metadata['tank_dimensions_c'] = pd.to_numeric(metadata['tank_dimensions_c'], errors='coerce').astype(float)

X = metadata[[
    'block_position_a',
    'block_position_b',
    'block_position_c',
    'block_dimensions_a',
    'block_dimensions_b',
    'block_dimensions_c',
    'fluid_volume',
    'block_velocity_magnitude',
    'density',
    'tank_dimensions_a',
     'tank_dimensions_b',
      'tank_dimensions_c',
    'time_max',
    'time_step',
    'particle_radius',
    'number_of_fluid_particles',

]]

y = metadata['kinematic_viscosity']





In [67]:
X

,block_position_a,block_position_b,block_position_c,block_dimensions_a,block_dimensions_b,block_dimensions_c,fluid_volume,block_velocity_magnitude,density,tank_dimensions_a,tank_dimensions_b,tank_dimensions_c,time_max,time_step,particle_radius,number_of_fluid_particles
0,0.001,0.485,0.402,0.502,0.339,0.260,0.044246,1.961099,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,945
1,0.198,0.091,0.394,0.522,0.211,0.494,0.054410,1.241693,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1050
2,0.040,0.277,0.401,0.692,0.216,0.427,0.063825,1.618311,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1260
3,0.211,0.250,0.405,0.673,0.155,0.416,0.043395,1.467080,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,720
4,0.195,0.149,0.117,0.548,0.625,0.190,0.065075,1.346985,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.052,0.116,0.362,0.525,0.442,0.201,0.046642,1.505146,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1017
96,0.379,0.068,0.363,0.204,0.725,0.453,0.066999,1.437174,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1430
97,0.439,0.395,0.432,0.217,0.387,0.480,0.040310,1.715568,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,770
98,0.010,0.367,0.224,0.706,0.464,0.166,0.054379,1.721226,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1131


In [68]:
y

0     0.000053
1     2.563858
2     0.080507
3     0.002160
4     1.754441
        ...   
95    0.000002
96    0.064882
97    0.000280
98    0.000001
99    0.083612
Name: kinematic_viscosity, Length: 100, dtype: float64

In [69]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   block_position_a           100 non-null    float64
 1   block_position_b           100 non-null    float64
 2   block_position_c           100 non-null    float64
 3   block_dimensions_a         100 non-null    float64
 4   block_dimensions_b         100 non-null    float64
 5   block_dimensions_c         100 non-null    float64
 6   fluid_volume               100 non-null    float64
 7   block_velocity_magnitude   100 non-null    float64
 8   density                    100 non-null    float64
 9   tank_dimensions_a          100 non-null    float64
 10  tank_dimensions_b          100 non-null    float64
 11  tank_dimensions_c          100 non-null    float64
 12  time_max                   100 non-null    float64
 13  time_step                  100 non-null    float64


In [78]:
X.isnull().sum()

block_position_a             0
block_position_b             0
block_position_c             0
block_dimensions_a           0
block_dimensions_b           0
block_dimensions_c           0
fluid_volume                 0
block_velocity_magnitude     0
density                      0
tank_dimensions_a            0
tank_dimensions_b            0
tank_dimensions_c            0
time_max                     0
time_step                    0
particle_radius              0
number_of_fluid_particles    0
dtype: int64

In [79]:
X.describe()

,block_position_a,block_position_b,block_position_c,block_dimensions_a,block_dimensions_b,block_dimensions_c,fluid_volume,block_velocity_magnitude,density,tank_dimensions_a,tank_dimensions_b,tank_dimensions_c,time_max,time_step,particle_radius,number_of_fluid_particles
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0,100.0,100.000,100.000,100.000000
mean,0.259770,0.259570,0.265020,0.399910,0.424250,0.407820,0.054224,1.581271,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1119.510000
std,0.158365,0.143437,0.135234,0.181565,0.160592,0.161393,0.008633,0.325903,0.0,0.0,0.0,0.0,0.0,0.000,0.000,224.612685
min,0.001000,0.002000,0.005000,0.154000,0.155000,0.156000,0.040113,0.489312,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,630.000000
25%,0.109750,0.133000,0.148500,0.266250,0.292500,0.258500,0.046929,1.361645,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,969.000000
50%,0.275500,0.275000,0.255000,0.350000,0.419500,0.403500,0.054395,1.670933,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1104.500000
75%,0.407500,0.381750,0.397750,0.490750,0.507000,0.518250,0.060976,1.849392,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1260.000000
max,0.497000,0.499000,0.498000,0.936000,0.826000,0.848000,0.069779,1.997443,1000.0,1.0,1.0,1.0,2.0,0.003,0.015,1584.000000


In [80]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   block_position_a           100 non-null    float64
 1   block_position_b           100 non-null    float64
 2   block_position_c           100 non-null    float64
 3   block_dimensions_a         100 non-null    float64
 4   block_dimensions_b         100 non-null    float64
 5   block_dimensions_c         100 non-null    float64
 6   fluid_volume               100 non-null    float64
 7   block_velocity_magnitude   100 non-null    float64
 8   density                    100 non-null    float64
 9   tank_dimensions_a          100 non-null    float64
 10  tank_dimensions_b          100 non-null    float64
 11  tank_dimensions_c          100 non-null    float64
 12  time_max                   100 non-null    float64
 13  time_step                  100 non-null    float64


In [95]:
X['block_position_a'].corr(y)

-0.05246980469961506

In [98]:
X=X.drop('block_position_a', axis=1)

In [96]:
X['block_position_b'].corr(y)

-0.12725228397398947

In [97]:
X['block_position_c'].corr(y)

-0.14250114291474067

In [99]:
X['block_dimensions_a'].corr(y)

0.0981890693699561

In [100]:
X['block_dimensions_b'].corr(y)

-0.022122005324019662

In [101]:
X=X.drop('block_dimensions_b', axis=1)

In [102]:
X['block_dimensions_c'].corr(y)

-0.06708965987582224

In [103]:
X=X.drop('block_dimensions_c', axis=1)

In [104]:
X['fluid_volume'].corr(y)

0.07787662963921532

In [106]:
X['block_velocity_magnitude'].corr(y)

-0.2162809120714883

In [107]:
X['density'].corr(y)

nan

In [108]:
X=X.drop('density', axis=1)

In [109]:
X['tank_dimensions_a'].corr(y)

nan

In [110]:
X=X.drop('tank_dimensions_a', axis=1)

In [111]:
X['tank_dimensions_b'].corr(y)

nan

In [112]:
X=X.drop('tank_dimensions_b', axis=1)

In [113]:
X['tank_dimensions_c'].corr(y)

nan

In [114]:
X=X.drop('tank_dimensions_c', axis=1)

In [117]:
X['time_max'].corr(y)

nan

In [118]:
X=X.drop('time_max', axis=1)

In [119]:
X['time_step'].corr(y)

nan

In [120]:
X=X.drop('time_step', axis=1)

In [121]:
X['particle_radius'].corr(y)

nan

In [86]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = Sequential()
model.add(Dense(15, input_dim=16, activation='relu'))
model.add(Dense(15, activation='relu'))
model.add(Dense(1, activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adam')

history = model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100
3/3 [==============================] - 1s 9ms/step - loss: 101717.2656
Epoch 2/100
3/3 [==============================] - 0s 7ms/step - loss: 68359.1406
Epoch 3/100
3/3 [==============================] - 0s 5ms/step - loss: 42825.5312
Epoch 4/100
3/3 [==============================] - 0s 6ms/step - loss: 23800.6914
Epoch 5/100
3/3 [==============================] - 0s 5ms/step - loss: 11325.2324
Epoch 6/100
3/3 [==============================] - 0s 6ms/step - loss: 4184.9404
Epoch 7/100
3/3 [==============================] - 0s 5ms/step - loss: 1432.3066
Epoch 8/100
3/3 [==============================] - 0s 5ms/step - loss: 1275.5802
Epoch 9/100
3/3 [==============================] - 0s 5ms/step - loss: 2145.4771
Epoch 10/100
3/3 [==============================] - 0s 5ms/step - loss: 3089.7109
Epoch 11/100
3/3 [==============================] - 0s 6ms/step - loss: 3510.8704
Epoch 12/100
3/3 [==============================] - 0s 6ms/step - loss: 3210.8284
Epoch 13/100
3/3 [=

In [87]:
y_pred= model.predict(X_test)

1/1 [==============================] - 0s 95ms/step


In [92]:

import pandas as pd
y_pred_readable = pd.DataFrame(y_pred, columns=['kinematic_viscosity'])

In [93]:
y_pred_readable

,kinematic_viscosity
0,-1.678944
1,2.845538
2,5.650114
3,5.691963
4,0.404285
5,-1.988762
6,2.998978
7,3.017705
8,3.705239
9,4.640550
